helper jupyter notebook file to generate more essays on the topic of electoral college. 

In [18]:
import openai
import numpy as np
import json
import pandas as pd
import seaborn as sns
from openai import OpenAI

client = OpenAI()

In [16]:


def generate_essays(prompt, filename, temperatures= np.linspace(0.25, 0.75, 51)):

    for i, temp in enumerate(temperatures):
        print(f"Generating essay {i+1}/{len(temperatures)} at temperature {temp}...")
        try:
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant writing an essay."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=1000,
                temperature=float(temp)
            )

            # Extract the generated essay text
            essay_text = response.choices[0].message.content
            # Append the response to a JSON file
            with open(filename, 'a') as file:
                json.dump({
                    'temperature': temp,
                    'essay': essay_text,
                    #'response_metadata': response
                }, file)
                file.write('\n')
        
        except Exception as e:
            print(f"Error at temperature {temp}: {e}")
            continue

# Example usage
generate_essays("Write an essay on the topic: 'Does the electoral college work?'", "./data/elect_gpt4o_essays.json")


Generating essay 1/51 at temperature 0.25...
Generating essay 2/51 at temperature 0.26...
Generating essay 3/51 at temperature 0.27...
Generating essay 4/51 at temperature 0.28...
Generating essay 5/51 at temperature 0.29...
Generating essay 6/51 at temperature 0.3...
Generating essay 7/51 at temperature 0.31...
Generating essay 8/51 at temperature 0.32...
Generating essay 9/51 at temperature 0.33...
Generating essay 10/51 at temperature 0.33999999999999997...
Generating essay 11/51 at temperature 0.35...
Generating essay 12/51 at temperature 0.36...
Generating essay 13/51 at temperature 0.37...
Generating essay 14/51 at temperature 0.38...
Generating essay 15/51 at temperature 0.39...
Generating essay 16/51 at temperature 0.4...
Generating essay 17/51 at temperature 0.41000000000000003...
Generating essay 18/51 at temperature 0.42000000000000004...
Generating essay 19/51 at temperature 0.43...
Generating essay 20/51 at temperature 0.44...
Generating essay 21/51 at temperature 0.45...


In [23]:
essaydf = pd.read_json("./data/elect_gpt4o_essays.json", lines=True)

In [24]:
essaydf['word_count'] = essaydf['essay'].apply(lambda x: len(x.split()))

In [30]:
evaldf = pd.read_pickle("./data/elect_evaldf.pkl")

In [31]:
evaldf.columns

Index(['index', 'text_substring', 'text', 'label', 'prompt_name', 'source',
       'RDizzl3_seven', 'model', 'embedding', 'text_embedded',
       'text_embedding_tc', 'number_of_grammatical_errors',
       'number_of_spelling_mistakes', 'grammatical_errors',
       'spelling_mistakes', 'sentence_complexity', 'lexical_diversity',
       'vocabulary_sophistication', 'coherence_score', 'cohesion_score',
       'argument_structure', 'readability_index', 'sentence_clarity',
       'engagement_score', 'sentiment_alignment', 'style_consistency',
       'relevance_score', 'evidence_support', 'originality_and_creativity',
       'likelihood_of_ai', 'reasoning_of_likelihood_of_ai',
       'substrings_of_high_likelihood_of_ai', 'Author'],
      dtype='object')

In [37]:
essaydf.rename(columns={'essay': 'text'}, inplace=True)

In [40]:
esaydf['Author'] = 'AI'
essaydf['source'] = 'GPT-4o'
essaydf['model'] = 'gpt'

In [ ]:
essaydf.to_pickle('./data/elect_gpt4o_essays.pkl')